<a href="https://colab.research.google.com/github/sobrinhodasilva/dados_conab/blob/main/SCRAPY_CONAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conab
Companhia Nacional de Abastecimento


Scrapy no site da CONAB para obtenção das planilhas com os referencial de custo de produdução agrícola, pecoaria e Sociobiodiversidade.

Mais inforamções em https://www.conab.gov.br/info-agro/custos-de-producao

In [5]:
# importing the libraries
from bs4 import BeautifulSoup
import requests
# Acessar os arquivos 
import os
#import regex 
import re
import pandas as pd
import json
# excell
import xlrd

In [6]:
url_base = "https://www.conab.gov.br"

In [7]:
# funcao de download arquivos 
def downloadFile(url, file_name):    
    r = requests.get(url, allow_redirects=True)
    open(file_name, 'wb').write(r.content)

In [8]:
# Abrir pagina web
def openWebPage(url):
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text
    # Parse the html content
    soup = BeautifulSoup(html_content, 'html.parser')
    #print(soup.prettify()) # print the parsed data of html
    return soup

In [9]:
# pagina de download 
def pagina_download (url_local):
    file = []
    op = openWebPage(url_local)
    # produra links com xls no nome 
    links = op.find_all("a", href = True, title = re.compile("xls"))
    # baixar todos os links da página 
    for link in links:
        file_name = (link["title"])        
        url_download = url_base + link["href"]
        file.append( {"file_name": file_name, "url_download" :url_download})          
    return(file)

## PLANILHAS DE CUSTOS DE PRODUÇÃO

In [10]:
url_pagina_entrada = url_base + "/info-agro/custos-de-producao/planilhas-de-custo-de-producao"
url_pagina_entrada

'https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao'

In [11]:
# abrir pagina que dará acesso a todas as subpáginas para cada  planilha de cultura 
pagina_entrada = openWebPage(url_pagina_entrada)

In [12]:
# Função que  procura dentro da tab da pagina todos os links para paginas de downloads de arquivos 
def lookUpTab(tab, folder):
    arquivos_baixados = []
      
    # Procurar os links na aba  
    for link in tab.find_all(href=True):   
        url = url_base + link['href']
        print("\n" + url_base + link['href'])
        arquivos = pagina_download(url)   
        
        #baixa todos os arquivos da página
        for arquivo in arquivos: 
            file_name = folder +"/"+ arquivo["file_name"]
            url= arquivo["url_download"]
            downloadFile(url , file_name)   
            print(file_name)
            arquivos_baixados.append({"file_name":file_name ,"folder": folder, "url":url })
        
    return arquivos_baixados
    #print(arquivo["url_download"])
   

In [13]:
# abrir cada aba 
abas = [{"div_id":"agrícolas-2", "folder":"agricola"}, {"div_id":"sociobiodiversidade-2" , "folder":"sociobiodiversidade" }, {"div_id":"pecu�rios-2", "folder":"pecuario"} ]
arquivos_baixados = []
for aba in abas:
    tab = pagina_entrada.find(id = aba["div_id"])
    folder = aba["folder"]
    try:
        os.makedirs(folder)
    except:
        print("pasta existe")        
    arquivos_baixados.append(lookUpTab(tab, folder ))


https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/787-abacaxi
agricola/Abacaxi_Serie_Historica_2008-2022.xlsx

https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/860-acai-agricolas
agricola/Acai_Serie_Historica_2016-2021.xlsx

https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/788-algodao-em-pluma
agricola/Algodao_Serie_Historica_1998-2022.xls

https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/789-alho
agricola/Serie_Historica_Alho_2015-2021.xlsx

https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/790-amendoim
agricola/Amendoim_Serie_Historica_2005-2022.xls

https://www.conab.gov.br/info-agro/custos-de-producao/planilhas-de-custo-de-producao/itemlist/category/791-arroz
agricola/Arroz_Sequeiro_Serie_Historica_2001-2022.xls
agric